In [2]:
#check python version
!python --version

Python 3.9.7


### Import Libraries

In [3]:
#import all necessary packages
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import time

### Read In Data File and Clean it

In [5]:
#read in udemy_df
udemy_df = pd.read_csv('udemy_master_df.csv', usecols = ['Title', 'Summary'])

In [6]:
#remove all null values
udemy_df = udemy_df[udemy_df['Summary'].notna()]

In [8]:
#create a copy of the udemy_df 
course_df = udemy_df.copy()

In [9]:
#check to make sure all null values have been removed
course_df.isnull().sum()

Title      0
Summary    0
dtype: int64

In [11]:
#drop all duplicated values
course_df  = course_df.drop_duplicates()

In [12]:
#remove all non-ascii characters from the dataset
course_df = course_df[course_df['Summary'].map(lambda x: x.isascii())]

In [13]:
#make all descriptions and titles lowercase.
course_df = course_df.apply(lambda x: x.str.lower())
job_df = job_df.apply(lambda x: x.str.lower())

In [14]:
#A contractions dictionary from Wikipedia found on Stack Overflow for expanding contractions: 
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions_dict = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [15]:
#This code is code to expand contractions in text created by Abhishek Sharma:
#https://www.analyticsvidhya.com/blog/2020/04/beginners-guide-exploratory-data-analysis-text-data/

#Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

#Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews and titles for course_df; job_df doesn't include any contractions
course_df['Summary']= course_df['Summary'].apply(lambda x:expand_contractions(x))
course_df['Title']= course_df['Title'].apply(lambda x:expand_contractions(x))

### Apply Keyword Extractor RAKE and Vectorize It Using CountVectorizer

In [16]:
#import rake for keyword extraction
from rake_nltk import Rake

In [17]:
#create an empty column for the keywords
course_df['keywords'] = ''

In [18]:
#Loop across all rows to extract all keywords from Summary
start_time = time.time()

for index, row in course_df.iterrows():
    summary = row['Summary']
    
    #instantiating Rake by default it uses English stopwords from NLTK and discards all punctuation chars
    r = Rake()
    
    #extract words by passing the text
    r.extract_keywords_from_text(summary)
    
    #get the dictionary with key words and their scores
    keyword_dict_scores = r.get_word_degrees()
    
    #assign keywords to new columns
    row['keywords'] = list(keyword_dict_scores.keys())
    
print("Time takes to extract keywords: ", round((time.time() - start_time)/60, 2), " mins")

Time takes to extract keywords:  0.44  mins


In [19]:
#let's see if RAKE extracted our keywords
course_df.head()

,Title,Summary,keywords
0,the complete sql bootcamp 2020: go from zero t...,become an expert at sql!,"[become, expert, sql]"
1,tableau 2020 a-z: hands-on tableau training fo...,learn tableau 2020 for data science step by st...,"[learn, tableau, 2020, data, science, step, re..."
2,pmp exam prep seminar - pmbok guide 6,pmp exam prep seminar - earn 35 pdus by comple...,"[pmp, exam, prep, seminar, earn, 35, pdus, com..."
3,the complete financial analyst course 2020,"excel, accounting, financial statement analysi...","[excel, accounting, financial, statement, anal..."
4,an entire mba in 1 course:award winning busine...,** #1 best selling business course! ** everyth...,"[**, 1, best, selling, business, course, every..."


### Get Similarity Between Courses

In [20]:
#import cosine similarity and countvectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
#create word vectors from course_df using countvectorizer 
count = CountVectorizer(max_df = 0.7,
                        max_features = 10000,
                        ngram_range = (1,3),
                        lowercase = False,
                        tokenizer= lambda i:i,
                       )
count_matrix = count.fit_transform(course_df['keywords'])

countvec_df = pd.DataFrame(count_matrix.toarray(), columns= count.get_feature_names())

countvec_df.index = course_df['Title']

In [23]:
#get similarity between courses, and create a cosine similarity df for recommender use
start_time = time.time()

cosine_similarity_array = cosine_similarity(countvec_df)

cosine_similarity_df = pd.DataFrame(cosine_similarity_array, columns=countvec_df.index, index=countvec_df.index)

print("Time takes to make cosine_similarity_df: ", round((time.time() - start_time)/60, 2), " mins")

Time takes to make cosine_similarity_df:  3.05  mins


In [24]:
#vectorize summary data for cosine similarity comparison against keywords
count2= CountVectorizer(
                        ngram_range = (1,3),
                        lowercase = False,
                       )

vectorized_data = count2.fit_transform(course_df['Summary'])

### Creating Recommender Model

In [26]:
#import functools to add a caching system to the recommender
import functools

@functools.lru_cache(maxsize=128)
def recommender(job_name,num):
    #create a vector of the inputted job title using the vector from above.
    query_vec = count2.transform([job_name])
    
    # calculate similarity with vectorized course summary
    similarity = cosine_similarity(query_vec, vectorized_data).flatten() 
    
    #create partitions in the data to make prediction faster
    indices = np.argpartition(similarity, -1)[-5:]
    
    #grab the course name that has the best similarity scores for the given job title
    course_name = course_df.iloc[indices][::-1]['Title'].values[0]
    
    #grab all matches to course names in cosine_similarity df
    cosine_similarity_series = cosine_similarity_df.loc[course_name]
    
    #order values in descending order
    ordered_similarities = cosine_similarity_series.sort_values(ascending=False)
    
    #take the top num of values in descending order
    course_list = ordered_similarities[0:num]
    
    return course_list

In [27]:
start_time = time.time()
print(recommender('Registered Nurse', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [28]:
start_time = time.time()
print(recommender('Translator', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [29]:
start_time = time.time()
print(recommender('Software Engineer', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [30]:
start_time = time.time()
print(recommender('Driver', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [31]:
start_time = time.time()
print(recommender('Retail Salesperson', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [32]:
start_time = time.time()
print(recommender('Project Manager', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [33]:
start_time = time.time()
print(recommender('DevOps Engineer', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [34]:
start_time = time.time()
print(recommender('Warehouse Associate', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [35]:
start_time = time.time()
print(recommender('Cashier', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [36]:
start_time = time.time()
print(recommender('Customer Service Representation', 5))
print("Time takes recommender to recommend: ", round((time.time() - start_time)/60, 2), " mins")

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64
Time takes recommender to recommend:  0.0  mins


In [51]:
recommender('Data Science', 5)

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64

In [52]:
recommender('Administrative Assistant', 5)

Title
the complete express.js developer course & create two sites    1.000000
backend development with php and perl                          0.516398
google kubernetes engine (gke) - a beginner's guide            0.516398
azure solution architect certification- az 300                 0.516398
learn how to assemble and paint beautiful miniatures           0.516398
Name: the complete express.js developer course & create two sites, dtype: float64